<a href="https://colab.research.google.com/github/RoshniUndhad/custom_odoo/blob/main/Weather_Research_Agent_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://home.openweathermap.org/api_keys

API key

In [ ]:
!pip install requests llama-index llama-index-llms-groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 9.1 MB/s eta 0:00:00


Importing the dependencies

In [ ]:
import os
import requests
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool

**Loading the GroQ LLM**

In [ ]:
os.environ["GROQ_API_KEY"] = "<your_groq_api_key>"

In [ ]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

In [ ]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "<your_owm_api_key>"

In [ ]:
city = "Chennai"
units = "metric"

In [ ]:
params = {
    "q": city,
    "appid": API_KEY,
    "units": units # use "imperial" for Fahrenheit
}

In [ ]:
response = requests.get(BASE_URL, params=params)

In [ ]:
print(response.json())

{'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 33.97, 'feels_like': 40.97, 'temp_min': 32.78, 'temp_max': 35.61, 'pressure': 1010, 'humidity': 67, 'sea_level': 1010, 'grnd_level': 1009}, 'visibility': 6000, 'wind': {'speed': 4.12, 'deg': 140}, 'clouds': {'all': 40}, 'dt': 1744611554, 'sys': {'type': 2, 'id': 2093220, 'country': 'IN', 'sunrise': 1744590402, 'sunset': 1744635075}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}


**Weather Forecast Function**

In [ ]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "<your_owm_api_key>"

# custom weather forecast function
def find_weather(city: str) -> dict:
  """This function returns the current weatehr forecast for the given city"""
  units = "metric"
  params = {
      'q': city,
      'appid': API_KEY,
      'units': units
  }
  response = requests.get(BASE_URL, params=params)
  result = response.json()

  return result

In [ ]:
# convert the function to a function tool
weather_tool = FunctionTool.from_defaults(fn=find_weather)

In [ ]:
tools_list = [weather_tool]

In [ ]:
agent = ReActAgent.from_tools(
    tools = tools_list,
    llm = llm,
    verbose = True
)

In [ ]:
response = agent.chat("What is the temperature like in Paris?")

> Running step f11a8be8-17b8-4fde-8587-64798dcf1ca2. Step input: What is the temperature like in Paris?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_weather
Action Input: {'city': 'Paris'}
Observation: {'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 8.29, 'feels_like': 7.15, 'temp_min': 6.32, 'temp_max': 9.52, 'pressure': 1009, 'humidity': 96, 'sea_level': 1009, 'grnd_level': 1000}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 60}, 'clouds': {'all': 100}, 'dt': 1744611974, 'sys': {'type': 2, 'id': 2012208, 'country': 'FR', 'sunrise': 1744606884, 'sunset': 1744655993}, 'timezone': 7200, 'id': 2988507, 'name': 'Paris', 'cod': 200}
> Running step 1498e960-cfef-4119-8ec2-6356f0d69f64. Step input: None
Thought: I have the current weather forecast for Paris, which includes the temperatur

In [ ]:
print(response.response)

The current temperature in Paris is 8.29 degrees Celsius, with a feels-like temperature of 7.15 degrees Celsius.


In [ ]:
response = agent.chat("What is the temperature like in Chennai?")

> Running step dc0a1ec4-5b13-4e3f-af93-e4b588ff5f99. Step input: What is the temperature like in Chennai?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_weather
Action Input: {'city': 'Chennai'}
Observation: {'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 33.86, 'feels_like': 40.86, 'temp_min': 33.34, 'temp_max': 35.57, 'pressure': 1009, 'humidity': 67, 'sea_level': 1009, 'grnd_level': 1009}, 'visibility': 6000, 'wind': {'speed': 4.12, 'deg': 140}, 'clouds': {'all': 40}, 'dt': 1744611980, 'sys': {'type': 2, 'id': 2093220, 'country': 'IN', 'sunrise': 1744590402, 'sunset': 1744635075}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}
> Running step cb6f6390-c83f-4138-b261-4f05e4643899. Step input: None
Thought: I have the weather information for Chennai. I can now provide th

In [ ]:
print(response.response)

The current temperature in Chennai is 33.86 degrees Celsius, with a feels-like temperature of 40.86 degrees Celsius.
